In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal

In [2]:
import matplotlib.pyplot as plt


In [3]:
def def_generator(latent_dims):
  w_init = RandomNormal(stddev = 0.02)
  model = Sequential()
  nodes = 7*7*256 #to randomly start to build a 7,7 image from the sample space
  model.add(Dense(nodes, kernel_initializer=w_init, input_dim = latent_dims))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Reshape((7, 7, 256)))

  model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding = 'same', kernel_initializer=w_init))
  model.add(BatchNormalization()) #(14, 14, 128)
  model.add(Activation('relu'))

  model.add(Conv2DTranspose(64, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer=w_init))
  model.add(BatchNormalization()) #(28, 28, 64)
  model.add(Activation('relu'))

  model.add(Conv2D(1, (7, 7), padding = 'same', kernel_initializer=w_init)) #reshapes it to (28, 28, 1)
  model.add(Activation('tanh'))
  return model

#No compilation since its not trained all alone , it is trained from the grads of the combined model


In [4]:
def def_discriminator(input_dims = (28, 28, 1)):
  w_init = RandomNormal(stddev = 0.02)
  model = Sequential()
  model.add(Conv2D(64, (4, 4), (2, 2), padding = 'same', 
                   kernel_initializer=w_init, input_shape = input_dims))
  model.add(BatchNormalization())
  model.add(LeakyReLU(0.2))

  model.add(Conv2D(128, (4, 4), (2, 2), padding = 'same', 
                   kernel_initializer=w_init))
  model.add(BatchNormalization())
  model.add(LeakyReLU(0.2))  

  model.add(Flatten())
  model.add(Dense(1, activation = 'linear', kernel_initializer = w_init))
  model.compile(loss = 'mse', optimizer = Adam(lr = 0.0002, beta_1 = 0.5))
  return model

In [5]:
def GAN_Model(generator, discriminator):
  #set the disc weights to frozen
  discriminator.trainable = False
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  model.compile(loss = 'mse', optimizer = Adam(lr = 0.0002, beta_1 = 0.5))
  return model

In [6]:
#since the generator gives out generated images by tanh, it is between -1 and 1
#hence the real samples need to be normalized too

def load_real_samples():
  (X_train, y_train), (X_test, y_test) = load_data()
  X = expand_dims(X_train, axis = -1)
  X = X.astype('float32')
  X = (X - 127.5)/127.5
  return X  

In [7]:
def generate_real_samples(data, num_samples): #this is to pick a batch of samples from training data

  idx = randint(0, data.shape[0], num_samples)
  X = data[idx]
  y = ones((num_samples, 1))
  return X, y

In [8]:
def latent_points(latent_dims, num_samples):
  x_in = randn(num_samples * latent_dims)
  x_in = x_in.reshape(num_samples, latent_dims)
  return x_in

In [9]:
def generate_fake_samples(generator, latent_dims, num_samples):
  x_in = latent_points(latent_dims, num_samples)
  X = generator.predict(x_in)
  y = zeros((num_samples, 1))
  return X, y

In [10]:
def summary(step, generator, latent_dims, num_samples = 100):
  X, = generate_fake_samples(generator, latent_dims, num_samples)
  X = (X+1)/2.0 #to plot it, renormalize to [0,1]
  for i in range(100):
    plt.subplot(10, 10, i + 1)
    plt.axis('off')
    plt.imshow(X[i, :, :, 0], cmap = 'gray_r') #reverse gray plot to ensure that op is black digits on a white bg

  #saving the whole plot
  file_name = 'generated_plot_%06d.png' % (step + 1)  
  plt.savefig(file_name)
  plt.close()

  file_name_model = 'model_%06d.h5' % (step + 1)
  generator.save(file_name_model)
  print('Saved the files %s and %s' % (file_name, file_name_model))

In [11]:
def plot_losses(d1, d2, g): #d1 is losses of D for real data, and d2 is for fake
  plt.plot(d1, label = 'dloss_1')
  plt.plot(d2, label = 'dloss_2')
  plt.plot(g, label = 'gloss_1')
  plt.legend()
  file_name = 'line_loss_plot.png'
  plt.savefig(file_name)
  plt.close()
  print('Saved the plot as %s' % (file_name))

In [12]:
# train the generator and discriminator
def train(generator, discriminator, GAN_model, dataset, latent_dims, n_epochs=30, n_batch=64):
	batches_per_epoch = int(dataset.shape[0] / n_batch)
	n_steps = batches_per_epoch * n_epochs
	half_batch = int(n_batch / 2)
	d1, d2, g = list(), list(), list()
 
	for i in range(n_steps):
		X_real, y_real = generate_real_samples(dataset, half_batch)
		X_fake, y_fake = generate_fake_samples(generator, latent_dims, half_batch)

		d_loss1 = discriminator.train_on_batch(X_real, y_real)
		d_loss2 = discriminator.train_on_batch(X_fake, y_fake)
  
		# update the generator via the discriminator
		z_input = latent_points(latent_dims, n_batch)
		y_real_g = ones((n_batch, 1))
		g_loss = GAN_model.train_on_batch(z_input, y_real_g)

		print('--%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))

		d1.append(d_loss1)
		d2.append(d_loss2)
		g.append(g_loss)
  
		# evaluate the model every epoch
		if (i+1) % (batches_per_epoch) == 0:
			summary(i, generator, latent_dims)

	plot_losses(d1, d2, g)

In [13]:
latent_dims = 100
discriminator = def_discriminator()
generator = def_generator(latent_dims)
GAN_model = GAN_Model(generator, discriminator)
data = load_real_samples()
print(data.shape)
train(generator, discriminator, GAN_model, data, latent_dims)

Streaming output truncated to the last 5000 lines.
--23125, d1=0.000, d2=0.000 g=0.085
--23126, d1=0.001, d2=0.000 g=0.094
--23127, d1=0.001, d2=0.000 g=0.088
--23128, d1=0.001, d2=0.000 g=0.092
--23129, d1=0.001, d2=0.000 g=0.088
--23130, d1=0.000, d2=0.000 g=0.089
--23131, d1=0.000, d2=0.001 g=0.097
--23132, d1=0.000, d2=0.000 g=0.087
--23133, d1=0.000, d2=0.000 g=0.097
--23134, d1=0.001, d2=0.000 g=0.087
--23135, d1=0.002, d2=0.000 g=0.098
--23136, d1=0.002, d2=0.000 g=0.092
--23137, d1=0.001, d2=0.000 g=0.100
--23138, d1=0.001, d2=0.000 g=0.097
--23139, d1=0.000, d2=0.000 g=0.092
--23140, d1=0.000, d2=0.001 g=0.097
--23141, d1=0.000, d2=0.001 g=0.089
--23142, d1=0.000, d2=0.001 g=0.103
--23143, d1=0.001, d2=0.001 g=0.087
--23144, d1=0.001, d2=0.001 g=0.100
--23145, d1=0.001, d2=0.000 g=0.087
--23146, d1=0.001, d2=0.000 g=0.102
--23147, d1=0.001, d2=0.000 g=0.085
--23148, d1=0.002, d2=0.000 g=0.098
--23149, d1=0.001, d2=0.000 g=0.088
--23150, d1=0.001, d2=0.000 g=0.095
--23151, d1=0

In [14]:
from keras.models import load_model

def plot_generated(data, n):
  for i in range(n*n):
    plt.subplot(n, n, i+1)
    plt.axis('off')
    plt.imshow(data[i, :, :, 0], cmap = 'gray_r')
  plt.show()    

In [16]:
model = load_model('/content/model_028110.png/saved_model.pb')
latent_pts = latent_points(100, 100)
X = model.predict(latent_pts)
plot_generated(X, 10)

OSError: ignored